# Домашнее задание 7. Градиентный бустинг над решающими деревьями. Регрессия. 

Дедлайн: 07.06.2020 23:59

При градиентном бустинге деревья обучаются итеративно. Каждое новое дерево обучается на ошибках предыдущего. 
Алгоритм обучения градиентного бустинга:
+ $T$ - количество деревьев
+ $\gamma$ - размер шага (learning_rate)
+ $\{ (x_i, y_i )\}_{i=1}^N$ - обучающая выборка   


1. Инициализировать массив предсказаний $prediction$ ансамбля, заполнив его нулями. 
2. For $t$ in $1...T$      
  2.1. Посчитать остатки - антиградиент функции ошибки.    
  ***Ошибка:*** $mse(y,prediction) = (y - prediction)^2$    
  ***Градиент:*** $\nabla_{prediction} mse(y,prediction) = prediction - y$   
  ***Антиградиент:*** $residuals_t = y - prediction$    
  2.2. Обучить дерево $b_t$, обучающая выборка:$ \{ (x_i, residuals_{t, i} ) \}_{i=1}^N$ 
  (в качестве целевой переменной выступают остатки)   
  2.3. Сделать предсказание обученным деревом:     
    $prediction_t = b_t(x)$    
  2.4. Прибавить предсказанние текущей модели умноженное на размер шага к вектору предсказаний ансамбля: $prediction \mathrel{+}= \gamma*prediction_t$    




Итоговый ансамбль имеет вид:  $a(x) = \sum_{t = 1}^T (\gamma b_t(x))$

       

Ваша задача - заполнить пропуски # YOUR CODE HERE, и выполнить код

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

Обязательно фиксируем random state. 

In [2]:
SEED = 22
np.random.seed(SEED)

Реализуем простой градиентный бустинг, в качестве базового алгоритма используем DecisionTreeRegressor из sklearn. 

In [3]:
class SimpleGBRegressor:

    def __init__(self, n_estimators=10, max_depth=5, 
                 min_samples_leaf=1, learning_rate=0.1, 
                 random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.trees = []
        self.was_fit = False

    def fit(self, X, y):
        # убедиться что в X и y одинаковое число элементов
        assert X.shape[0] == y.shape[0]
        
        # инициализировать массив с предсказаниями, заполнив нулями
        prediction = np.zeros([y.shape[0]])

        # обучаем деревья
        for i in range(self.n_estimators):
            # посчитать остатки
            residual = y - prediction

            # инициализировать дерево с нужными параметрами
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                         min_samples_leaf = self.min_samples_leaf,
                                        random_state=self.random_state)

            # обучить дерево
            tree.fit(X, residual)

            # сделать предсказание текущего дерева
            tree_prediction = tree.predict(X)

            # сохранить обученное дерево
            self.trees.append(tree)

            # обновить вектор предсказаний модели
            prediction += self.learning_rate * tree_prediction

        self.was_fit = True
        return self

    def predict(self, X):
        # если модель не была обучена, печатаем сообщение об этом и вовращаем None
        if not self.was_fit:
            print("Model was not fitted")
            return None

        # инициализировать массив с предсказаниями
        y_pred = np.zeros(X.shape[0])
        
        # добавить прогнозы деревьев
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)

        # убедиться что в X и y одинаковое число элементов
        assert X.shape[0] == y_pred.shape[0]
        return y_pred

## Провеярем нашу модель


Загружаем датасет Diabetes. 

In [4]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
diabetes = load_diabetes()

In [5]:
# print(diabetes['DESCR'])

data = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
target = diabetes.target

Делим на обучающую и тестовую выборку, размер тестовой выборки - 30%. Не забываем про random state.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target,test_size = 0.3,
                                                    random_state = SEED)

Обучаем нашу модель и делаем предсказание по тестовой выборке. 
Параметры модели:
+ n_estimators = 100
+ random_state - зафиксированный нами ранее
+ остальные значения по умолчанию 

In [7]:
# создаем и обучаем модель
my_model = SimpleGBRegressor(n_estimators=100, random_state=SEED)
my_model.fit(X_train, y_train)

In [8]:
# делаем предсказание по тестовой выборке
y_pred = my_model.predict(X_test)

Считаем MSE

In [9]:
# оцениваем качество
mse = MSE(y_test, y_pred)
print(mse)

3749.674167308474


Сравниваем с реализацией аналогичного алгоритма из sklearn c **такими же** параметрами (будьте внимательны)

In [10]:
# создаем и обучаем модель
sklearn_model = GradientBoostingRegressor(n_estimators=100, max_depth=5,
                                           min_samples_leaf=1, learning_rate=0.1,
                                           random_state=SEED)
sklearn_model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=22, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
# делаем предсказание по тестовой выборке
y_pred_sklearn = sklearn_model.predict(X_test)

In [12]:
# оцениваем качество
mse = MSE(y_test, y_pred_sklearn)
print(mse)

3725.914043073783
